In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pymesh
#https://pymesh.readthedocs.io/en/latest/basic.html


import meshplot
# for display of meshes
#https://skoch9.github.io/meshplot/tutorial/

import random


### code for BYORP calculation

The radiation force from the $i$-th facet is
$$ {\bf F}_i = - \frac{\Phi}{c} {S_i} (\hat {\bf n}_i \cdot \hat {\bf s}) \hat {\bf n}_i $$
where  $S_i$ is the area of the $i$-th facet and $\hat {\bf n}_i$ is its surface normal.
Here $\Phi$ is the solar flux and $c$ is the speed of light.
The direction of the Sun is $\hat {\bf s}$.

The total Yarkovsky force is a sum over all the facets 
$${\bf F}_Y = \sum_{i: \hat {\bf n}_i \cdot \hat {\bf s} >0} {\bf F}_i$$

Only facets on the day side  or with $\hat {\bf n}_i \cdot \hat {\bf s} >0$ 
are included in the sum.

${\bf F}_Y$ is the instantaneous Yarkovsky force.

The Torque affecting the binary orbit is
$$ {\bf T} = \int dt\ {\bf a} \times {\bf F}_Y $$
where ${\bf a}$ is the secondary's radial vector from the binary center of mass.
If $\hat {\bf l}$ is the binary orbit normal then 
$$ {\bf T} \cdot \hat {\bf l} $$ 
changes the orbital angular momentu and  causes binary orbit migration.



In [4]:
# perturb a sphere (mesh, premade) and stretch it so that
# it becomes an ellipsoid.  
#    We can't directly edit vertices or faces
#    see this:  https://github.com/PyMesh/PyMesh/issues/156
#    the work around is to copy the entire mesh after modifying it
# Randomly add devrand to x,y,z positions of each vertex
# stretch or compress by aratio1 and aratio2 
# return a new mesh
def sphere_perturb(sphere,devrand,aratio1,aratio2):
    #devrand = 0.05  # how far to perturb each vertex
    nv = len(sphere.vertices)
    f = sphere.faces
    v = np.copy(sphere.vertices)
    for i in range(nv):
        dx = devrand*random.uniform(-1,1)
        dy = devrand*random.uniform(-1,1)
        dz = devrand*random.uniform(-1,1)
        v[i,2] *= aratio1 # 0.9  # make oblate, adjusts z
        v[i,1] *= aratio2 # 1.2  # make elongated, adjusts y
        v[i,0] += dx
        v[i,1] += dy
        v[i,2] += dz
        sub_com(v)
    psphere = pymesh.form_mesh(v, f)
    psphere.add_attribute("face_area")
    psphere.add_attribute("face_normal")
    return psphere
    

# substract the center of mass from a list of vertices
def sub_com(v):
    nv = len(v)
    xsum = np.sum(v[:,0])
    ysum = np.sum(v[:,1])
    zsum = np.sum(v[:,2])
    xmean = xsum/nv
    ymean = ysum/nv
    zmean = zsum/nv
    v[:,0]-= xmean 
    v[:,1]-= ymean 
    v[:,2]-= ymean 
    

In [5]:
# compute the radiation force instantaneously on a triangular mesh
# s_hat is a 3 length np.array (a unit vector) pointing to the Sun
# return Force vector (without Phi/c factor)
def F_Y(mesh,s_hat):
    s_len = np.sqrt(s_hat[0]**2 + s_hat[1]**2 + s_hat[2]**2)  # in case not normalized
    #nf = len(mesh.faces)
    S_i = mesh.get_face_attribute('face_area')  # vector of facet areas
    f_normal = mesh.get_face_attribute('face_normal')  # vector of vector of facet normals
    # normal components
    nx = np.squeeze(f_normal[:,0])    # a vector
    ny = np.squeeze(f_normal[:,1])
    nz = np.squeeze(f_normal[:,2])
    # dot product of n_i and s_hat
    n_dot_s = (nx*s_hat[0] + ny*s_hat[1] + nz*s_hat[2])/s_len  # a vector
    F_i_x = -S_i*n_dot_s*nx #  a vector
    F_i_y = -S_i*n_dot_s*ny
    F_i_z = -S_i*n_dot_s*nz
    ii = (n_dot_s >0)  # the day side only 
    # sum only day lit facets
    F_x = np.sum(F_i_x[ii]) # a number
    F_y = np.sum(F_i_y[ii])
    F_z = np.sum(F_i_z[ii])
    F_vec = np.zeros(3)  # the force vector
    F_vec[0] = F_x;  F_vec[1] = F_y; F_vec[2] = F_z
    #return F_x,F_y,F_z # return force
    return F_vec
    
#test       
#s_hat = np.array([0,1,0])  
#F_x,F_y,F_z = F_Y(psphere,s_hat)  
#print(F_x,F_y,F_z)


# compute cross product C=AxB
def cross_prod(A,B):
    C = np.zeros(3)
    Cx = A[1]*B[2] - A[2]*B[1]
    Cy = A[2]*B[0] - A[0]*B[2]
    Cz = A[0]*B[1] - A[1]*B[0]
    C[0] = Cx; C[1]= Cy; C[2] = Cz
    return C

In [6]:
# first rotate vertices in the mesh about the z axis by angle phi in radians
# then tilt over the body by obliquity which is an angle in radians
#     this tilts the z axis, and rotates about y axis by angle obliquity
# returns: 
#     the tilted mesh
#     the new z-body axis
def tilt_obliq(mesh,obliquity,phi):
    f = mesh.faces
    v = np.copy(mesh.vertices)
    nv = len(v)
    axis1 = np.array([0,0,1]) # z axis
    q1 = pymesh.Quaternion.fromAxisAngle(axis1, phi)
    axis2 = np.array([0,1,0]) # y axis 
    q2 = pymesh.Quaternion.fromAxisAngle(axis2, obliquity)
    for i in range(nv):
        v[i] = q1.rotate(v[i])
        v[i] = q2.rotate(v[i])
    
    new_mesh = pymesh.form_mesh(v, f)
    new_mesh.add_attribute("face_area")
    new_mesh.add_attribute("face_normal")
    zaxis = np.array([0,0,1])
    zrot = q2.rotate(zaxis) # body principal axis
    return new_mesh,zrot
    

In [7]:
# create a sphere of radius 1
center = np.array([0,0,0])
sphere = pymesh.generate_icosphere(1., center, refinement_order=2)
sphere.add_attribute("face_area")
sphere.add_attribute("face_normal")
#mesh.add_attribute("face_centroid")
#meshplot.plot(sphere.vertices, sphere.faces)

In [28]:
# create a perturbed ellipsoid using the above sphere
devrand = 0.05  # perturbation size
aratio1 = 0.5   # axis ratios
aratio2 = 0.7
body = sphere_perturb(sphere,devrand,aratio1,aratio2)  # create it
p=meshplot.plot(body.vertices, body.faces,return_plot=True)  # show it
# add a red line which could show where the binary is
r = 1.5; theta = np.pi/4
p0 = np.array([0,0,0]); p1 = np.array([r*np.cos(theta),r*np.sin(theta),0])
p.add_lines(p0, p1, shading={"line_color": "red", "line_width": 1.0}); 

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.012445…

In [29]:
# return a body rotated and tilted after iphi rotations by dphi
# also return binary direction assuming same rotation rate
# and return binary angular momentum orbital axis
# for tidally locked setting, circular orbit
def tilt_and_bin(body,obliquity,nphi,iphi):
    dphi = 2*np.pi/nphi
    phi = iphi*dphi
    tbody,zrot = tilt_obliq(body,obliquity,phi)
    a_bin = np.array([np.cos(phi),np.sin(phi),0.0])   # direction to binary
    l_bin = np.array([0,0,1.0])  # angular momentum axis of binary orbit
    return tbody,a_bin,l_bin

In [35]:
# test here looks okay!
# rotate body and show binary direction at the same time.
tbody,a_bin,l_bin = tilt_and_bin(body,0,20,4)
p=meshplot.plot(tbody.vertices, tbody.faces,return_plot=True)
# add a line which could show where the binary is
r = 1.5; 
p0 = np.array([0,0,0]); p1 = 1.5*a_bin
p.add_lines(p0, p1, shading={"line_color": "red", "line_width": 1.0}); 

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.006743…

In [88]:
# tilt the object to check the rotation routine, test passed
obliquity = 5.*np.pi/180.
phi = 60.*np.pi/180
tsphere,zrot = tilt_obliq(psphere2,obliquity,phi)
#meshplot.plot(tsphere.vertices, tsphere.faces)
#print(zrot)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0034860…

[0.08715574 0.         0.9961947 ]
